In [1]:
import pandas as pd
import numpy as np
from rep.utils import train_test_split_group
from sklearn.cross_validation import train_test_split
import xgboost as xgb
from scipy.stats import kurtosis,skew, mode
import matplotlib.pyplot as plt
plt.style.use('ggplot')
%matplotlib

Using matplotlib backend: Qt5Agg


/home/dz/anaconda3/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [ ]:
train = pd.read_csv("data/merchants_train.csv", sep = ';')
train.columns = ['merchant_id', 'x', 'y']
tranz = pd.read_csv("data/transactions.csv")
test = pd.read_csv("data/merchants_test.csv", sep = ';')#, encoding = 'Windows-1251',sep = ";")
test.columns = ['merchant_id', 'x', 'y']
Xtr = pd.merge(tranz,train,on='merchant_id')
Xtest = pd.merge(test,tranz,on='merchant_id')
data_tr, data_val = train_test_split_group(Xtr.merchant_id, Xtr, random_state=7, train_size=0.8)
data_tr['Test'] = 0
data_val['Test'] = 1
Xtest['Test'] = 2
print(len(data_tr),len(data_val),len(Xtest),len(train),len(test),len(tranz))
tranz = pd.concat([data_tr,data_val,Xtest])
print(len(tranz))
tranz.head()

In [ ]:
def bol(x):
    if x<0.002:
        return 1
    else:
        return 0

tranz['x_p'] = abs(tranz['x']-tranz['latitude'])
tranz['x_p'] = tranz['x_p'].apply(bol)
tranz['y_p'] = abs(tranz['y']-tranz['longitude'])
tranz['y_p'] = tranz['y_p'].apply(bol)
tranz['cool'] = tranz['x_p']*tranz['y_p']
tranz.drop(['x_p','y_p'],inplace = True,axis=1)

In [ ]:
''''
msk = df['latitude']>30
X = df[['latitude','merchant_id']][msk].groupby('merchant_id').median()
X.columns = ['latitude_median']
df = pd.merge(df,X,how = 'left', left_on='merchant_id',right_index=True)
msk = df['latitude']<30
df.loc[msk,'latitude'] = df.loc[msk,'latitude_median']
df

In [ ]:
def toch(x):
    return "%.5f" % x
#tranz['latitude'] = tranz['latitude'].apply(toch)
#tranz['longitude'] = tranz['longitude'].apply(toch)

In [ ]:
tranz['merchant_id'] = tranz['merchant_id'].apply(int)
#tranz['latitude'] = tranz['latitude'].apply(toch)
#tranz['longitude'] = tranz['longitude'].apply(toch)

In [ ]:
tranz['num'] = tranz.groupby('merchant_id')['latitude'].transform('count')
def nol(x):
    if x == 0:
        return 1
    else:
        return 0
tranz['0'] = tranz['latitude'].apply(nol)
tranz['0'] = tranz.groupby('merchant_id')['0'].transform('sum')

In [ ]:
def divide(a, b):
    if b>0:
        return a/b
    else:
        return 8000#!!!!!!!!!!!!!!!!!!!!!!!!!!
tranz['div'] = tranz.apply(lambda row: divide(row['num'], row['0']), axis=1)
tranz['delta_num_0'] = tranz['num'] - tranz['0']

In [ ]:
tranz['countlat']=tranz.groupby(['latitude'])['latitude'].transform('count')
tranz['countlong']=tranz.groupby(['longitude'])['longitude'].transform('count')

In [ ]:
tranz['div_count_lat'] = tranz.apply(lambda row: divide(row['num'], row['countlat']), axis=1)
tranz['div_count_long'] = tranz.apply(lambda row: divide(row['num'], row['countlong']), axis=1)

In [ ]:
tranz['div_count_lat_0_all'] = tranz.apply(lambda row: divide(row['0'], row['countlat']), axis=1)
tranz['div_count_long_0_all'] = tranz.apply(lambda row: divide(row['0'], row['countlong']), axis=1)

In [ ]:
dates = pd.to_datetime(tranz['real_transaction_dttm'])
tranz['Hourtr'] = dates.apply(lambda x: x.hour)
tranz['Hourminutetr'] = dates.apply(lambda x: x.hour + x.minute/60)
tranz['minutetr'] = dates.apply(lambda x: x.minute)
tranz['secondtr'] = dates.apply(lambda x: x.second)

In [ ]:
dates = pd.to_datetime(tranz['record_date'])
tranz['Hourrec'] = dates.apply(lambda x: x.hour)
tranz['Hourminuterec'] = dates.apply(lambda x: x.hour + x.minute/60)
tranz['minuterec'] = dates.apply(lambda x: x.minute)
tranz['secondrec'] = dates.apply(lambda x: x.second)

In [ ]:
tranz['str'] = tranz['Hourtr']*3600+tranz['minutetr']*60+tranz['secondtr']
tranz['srec'] = tranz['Hourrec']*3600+tranz['minuterec']*60+tranz['secondrec']
tranz['sdiff'] =  tranz['srec'] - tranz['str']

In [ ]:
def times00(x):
    if x > 3600:
        return -86400 + x
    elif x<-3600:
        return x + 86400
    else:
        return x
tranz['sdiff'] = tranz['sdiff'].apply(times00)


In [ ]:
def dif_pos(x):
    if x>0:
        return 1
    else:
        return 0
tranz['dif_pos'] = tranz['sdiff'].apply(dif_pos)

In [ ]:
def dif_time_10(x):
    if -10<x<10:
        return 1
    else:
        return 0
tranz['dif_10'] = tranz['sdiff'].apply(dif_time_10)
def dif_time_20(x):
    if -20<x<20:
        return 1
    else:
        return 0
tranz['dif_20'] = tranz['sdiff'].apply(dif_time_20)
def dif_time_30(x):
    if -30<x<30:
        return 1
    else:
        return 0
tranz['dif_30'] = tranz['sdiff'].apply(dif_time_30)
def dif_time_40(x):
    if -40<x<40:
        return 1
    else:
        return 0
tranz['dif_40'] = tranz['sdiff'].apply(dif_time_40)
def dif_time_50(x):
    if -50<x<50:
        return 1
    else:
        return 0
tranz['dif_50'] = tranz['sdiff'].apply(dif_time_50)
def dif_time_60(x):
    if -60<x<60:
        return 1
    else:
        return 0
tranz['dif_60'] = tranz['sdiff'].apply(dif_time_60)
def dif_time_120(x):
    if -120<x<120:
        return 1
    else:
        return 0
tranz['dif_120'] = tranz['sdiff'].apply(dif_time_120)
def dif_time_300(x):
    if -300<x<300:
        return 1
    else:
        return 0
tranz['dif_300'] = tranz['sdiff'].apply(dif_time_300)
def dif_time_600(x):
    if -600<x<600:
        return 1
    else:
        return 0
tranz['dif_600'] = tranz['sdiff'].apply(dif_time_600)

In [ ]:
def dif_time_10(x):
    if -10<x<10:
        return 1
    else:
        return 0
tranz['dif_10_in'] = tranz['sdiff'].apply(dif_time_10)
def dif_time_20(x):
    if (-20<x<-10)or(10<x<20):
        return 1
    else:
        return 0
tranz['dif_20_in'] = tranz['sdiff'].apply(dif_time_20)
def dif_time_30(x):
    if (-30<x<-20)or(20<x<30):
        return 1
    else:
        return 0
tranz['dif_30_in'] = tranz['sdiff'].apply(dif_time_30)
def dif_time_40(x):
    if (-40<x<-30)or(30<x<40):
        return 1
    else:
        return 0
tranz['dif_40_in'] = tranz['sdiff'].apply(dif_time_40)
def dif_time_50(x):
    if (-40<x<-30)or(30<x<40):
        return 1
    else:
        return 0
tranz['dif_50_in'] = tranz['sdiff'].apply(dif_time_50)
def dif_time_60(x):
    if (-50<x<-40)or(40<x<50):
        return 1
    else:
        return 0
tranz['dif_60_in'] = tranz['sdiff'].apply(dif_time_60)
def dif_time_120(x):
    if (-120<x<-60)or(60<x<120):
        return 1
    else:
        return 0
tranz['dif_120_in'] = tranz['sdiff'].apply(dif_time_120)
def dif_time_300(x):
    if (-300<x<-120)or(120<x<300):
        return 1
    else:
        return 0
tranz['dif_300_in'] = tranz['sdiff'].apply(dif_time_300)
def dif_time_600(x):
    if (-600<x<-300)or(300<x<600):
        return 1
    else:
        return 0
tranz['dif_600_in'] = tranz['sdiff'].apply(dif_time_600)

In [ ]:
tranz.drop(['real_transaction_dttm','record_date'],axis=1, inplace = True)

In [ ]:
tranz['count_mecrh_lat']=tranz.groupby(['merchant_id','latitude'])['latitude'].transform('size')
tranz['count_mecrh_long']=tranz.groupby(['merchant_id','longitude'])['longitude'].transform('size')

In [ ]:
tranz['delta_count'] = tranz['countlat']-tranz['countlong']

In [ ]:
tranz['delta_count_num_lat'] = tranz['num']-tranz['count_mecrh_lat']
tranz['delta_count_num_long'] = tranz['num']-tranz['count_mecrh_long']

In [ ]:
tranz['div_count_n_lat'] = tranz.apply(lambda row: divide(row['num'], row['count_mecrh_lat']), axis=1)
tranz['div_count_n_long'] = tranz.apply(lambda row: divide(row['num'], row['count_mecrh_long']), axis=1)

In [ ]:
tranz['div_count_all_lat'] = tranz.apply(lambda row: divide(row['count_mecrh_lat'], row['countlat']), axis=1)
tranz['div_count_all_long'] = tranz.apply(lambda row: divide(row['count_mecrh_long'], row['countlong']), axis=1)

In [ ]:
tranz['latitude'] = tranz['latitude'].apply(toch)
tranz['longitude'] = tranz['longitude'].apply(toch)

In [ ]:
tranz['count_mecrh_lat_toch']=tranz.groupby(['latitude'])['latitude'].transform('size')
tranz['count_mecrh_long_toch']=tranz.groupby(['longitude'])['longitude'].transform('size')

In [ ]:

tranz['delta_count_toch_lat'] = tranz['countlat']-tranz['count_mecrh_lat_toch']
tranz['delta_count_toch_long'] = tranz['countlat']-tranz['count_mecrh_lat_toch']

In [ ]:
print(len(tranz))
#первые остаются
#t = tranz[tranz.duplicated(['latitude','longitude'],take_last=True)]#,keep = True)]
tranz = tranz.drop_duplicates(cols=['merchant_id','latitude','longitude'], take_last=True)


In [ ]:
tranz.head()

In [ ]:
print(len(tranz))

# drop_count_high

In [ ]:
tranz['latitude'] = tranz['latitude'].apply(float)
tranz['longitude'] = tranz['longitude'].apply(float)

In [ ]:
1.15/0.0019

In [ ]:
520*0.0019

In [ ]:
x= 525*np.arange(-0.2,0.2,0.0001)
plt.plot(x,1/(1+np.exp(-(x)**20))-0.5)
plt.axis([-3,3,-0.001,2])
plt.show()

In [ ]:
tranz['countlat_drop']=tranz.groupby(['latitude'])['latitude'].transform('count')
tranz['countlong_drop']=tranz.groupby(['longitude'])['longitude'].transform('count')
tranz['delta_drop'] = tranz['countlat_drop']-tranz['countlong_drop']
tranz['delta_drop_lat'] = tranz['countlat']-tranz['countlat_drop']
tranz['delta_drop_long'] = tranz['countlong']-tranz['countlong_drop']
#div

In [ ]:
tranz['delta_drop_lat_toch'] = tranz['count_mecrh_lat_toch']-tranz['countlat_drop']
tranz['delta_drop_long_toch'] = tranz['count_mecrh_lat_toch']-tranz['countlong_drop']

In [ ]:
tranz = tranz[tranz['latitude'] !=0]

In [ ]:
print(len(tranz))

In [ ]:
tranz['count_mecrh_lat_drop_0']=tranz.groupby(['merchant_id','latitude'])['latitude'].transform('size')
tranz['count_mecrh_long_drop_0']=tranz.groupby(['merchant_id','longitude'])['longitude'].transform('size')
tranz['count_mecrh_lat_diff_0'] = tranz['count_mecrh_lat'] - tranz['count_mecrh_lat_drop_0']
tranz['count_mecrh_long_diff_0'] = tranz['count_mecrh_long'] - tranz['count_mecrh_long_drop_0']

In [ ]:
tranz['count_mecrh_lat_drop_0_all']=tranz.groupby(['latitude'])['latitude'].transform('size')
tranz['count_mecrh_long_drop_0_all']=tranz.groupby(['longitude'])['longitude'].transform('size')
tranz['count_mecrh_lat_diff_0_all'] = tranz['countlat'] - tranz['count_mecrh_lat_drop_0_all']
tranz['count_mecrh_long_diff_0_all'] = tranz['countlong'] - tranz['count_mecrh_long_drop_0_all']

In [ ]:

tranz['count_mecrh_lat_diff_0_all_toch'] = tranz['count_mecrh_lat_toch'] - tranz['count_mecrh_lat_drop_0_all']
tranz['count_mecrh_long_diff_0_all_toch'] = tranz['count_mecrh_long_toch'] - tranz['count_mecrh_long_drop_0_all']

In [ ]:
tranz = tranz[tranz['latitude'] >=42]######################>40
print(len(tranz))
tranz = tranz[tranz['latitude'] <=70]
print(len(tranz))
tranz = tranz[tranz['longitude'] >=19]
print(len(tranz))
tranz = tranz[tranz['longitude'] <=170]
print(len(tranz))

In [ ]:
train['x'].min(),train['x'].max(),train['y'].min(),train['y'].max()

In [ ]:
print(len(tranz))

In [ ]:
tranz['count_mecrh_lat_drop_10']=tranz.groupby(['merchant_id','latitude'])['latitude'].transform('size')
tranz['count_mecrh_long_drop_10']=tranz.groupby(['merchant_id','longitude'])['longitude'].transform('size')
tranz['count_mecrh_lat_drop_10_all']=tranz.groupby(['latitude'])['latitude'].transform('size')
tranz['count_mecrh_long_drop_10_all']=tranz.groupby(['longitude'])['longitude'].transform('size')

In [ ]:
tranz['count_mecrh_lat_diff_10'] = tranz['count_mecrh_lat'] - tranz['count_mecrh_lat_drop_10']
tranz['count_mecrh_long_diff_10'] = tranz['count_mecrh_long'] - tranz['count_mecrh_long_drop_10']
tranz['count_mecrh_lat_diff_0_10'] = tranz['count_mecrh_lat_drop_0'] - tranz['count_mecrh_lat_drop_10']
tranz['count_mecrh_long_diff_0_10'] = tranz['count_mecrh_long_drop_0'] - tranz['count_mecrh_long_drop_10']

In [ ]:
tranz['count_mecrh_lat_diff_10_all'] = tranz['countlat'] - tranz['count_mecrh_lat_drop_10_all']
tranz['count_mecrh_long_diff_10_all'] = tranz['countlong'] - tranz['count_mecrh_long_drop_10_all']
tranz['count_mecrh_lat_diff_0_10_all'] = tranz['count_mecrh_lat_drop_0_all'] - tranz['count_mecrh_lat_drop_10_all']
tranz['count_mecrh_long_diff_0_10_all'] = tranz['count_mecrh_long_drop_0_all'] - tranz['count_mecrh_long_drop_10_all']

In [ ]:
tranz['count_mecrh_lat_diff_10_all_drop'] = tranz['countlat_drop'] - tranz['count_mecrh_lat_drop_10']
tranz['count_mecrh_long_diff_10_all_drop'] = tranz['countlong_drop'] - tranz['count_mecrh_long_drop_10']
tranz['count_mecrh_lat_diff_0_10_all_toch'] = tranz['count_mecrh_lat_toch'] - tranz['count_mecrh_lat_drop_10']
tranz['count_mecrh_long_diff_0_10_all_toch'] = tranz['count_mecrh_long_toch'] - tranz['count_mecrh_long_drop_10']

In [ ]:
tranz['num_drop'] = tranz.groupby('merchant_id')['latitude'].transform('count')
tranz['diff_num_drop'] = tranz['num'] - tranz['num_drop']

In [ ]:
tranz.head()

In [ ]:
tranz['kurt_lat']=tranz.groupby(['merchant_id'])['latitude'].transform(kurtosis)
tranz['kurt_long']=tranz.groupby(['merchant_id'])['longitude'].transform(kurtosis)

In [ ]:
tranz['skew_lat']=tranz.groupby(['merchant_id'])['latitude'].transform(skew)
tranz['skew_long']=tranz.groupby(['merchant_id'])['longitude'].transform(skew)

In [ ]:
tranz['mode_lat']=tranz.groupby(['merchant_id'])['latitude'].transform(mode)
tranz['mode_long']=tranz.groupby(['merchant_id'])['longitude'].transform(mode)

In [ ]:
tranz['mecrh_lat_std']=tranz.groupby(['merchant_id'])['latitude'].transform('std')
tranz['mecrh_long_std']=tranz.groupby(['merchant_id'])['longitude'].transform('std')

In [ ]:
tranz['mecrh_lat_sem']=tranz.groupby(['merchant_id'])['latitude'].transform('sem')
tranz['mecrh_long_sem']=tranz.groupby(['merchant_id'])['longitude'].transform('sem')

In [ ]:
tranz['mecrh_lat_max']=tranz.groupby(['merchant_id'])['latitude'].transform('max')
tranz['mecrh_long_max']=tranz.groupby(['merchant_id'])['longitude'].transform('max')

In [ ]:
tranz['mecrh_lat_min']=tranz.groupby(['merchant_id'])['latitude'].transform('min')
tranz['mecrh_long_min']=tranz.groupby(['merchant_id'])['longitude'].transform('min')

In [ ]:
tranz['max+min_lat'] = (tranz['mecrh_lat_max'] + tranz['mecrh_lat_min'])*0.5
tranz['max+min_long'] = (tranz['mecrh_long_max'] + tranz['mecrh_long_min'])*0.5

In [ ]:
tranz['max-min_lat'] = (tranz['mecrh_lat_max'] - tranz['mecrh_lat_min'])
tranz['max-min_long'] = (tranz['mecrh_long_max'] - tranz['mecrh_long_min'])

In [ ]:
tranz['mecrh_lat_median']=tranz.groupby(['merchant_id'])['latitude'].transform('median')
tranz['mecrh_long_median']=tranz.groupby(['merchant_id'])['longitude'].transform('median')

In [ ]:
tranz['mecrh_lat_first']=tranz.groupby(['merchant_id'])['latitude'].transform('first')
tranz['mecrh_long_first']=tranz.groupby(['merchant_id'])['longitude'].transform('first')

In [ ]:
tranz['mecrh_lat_last']=tranz.groupby(['merchant_id'])['latitude'].transform('last')
tranz['mecrh_long_last']=tranz.groupby(['merchant_id'])['longitude'].transform('last')

In [ ]:
tranz['mecrh_lat_mean']=tranz.groupby(['merchant_id'])['latitude'].transform('mean')
tranz['mecrh_long_mean']=tranz.groupby(['merchant_id'])['longitude'].transform('mean')

In [ ]:
for i in range(5,100,5):
    print(i)
    tranz['mecrh_lat_perc_{}'.format(i)] = tranz.groupby(['merchant_id'])['latitude'].transform(lambda x: np.percentile(x, i))
    tranz['mecrh_long_perc_{}'.format(i)] = tranz.groupby(['merchant_id'])['longitude'].transform(lambda x: np.percentile(x, i))
    tranz['diff_lat_mecrh_lat_perc_{}'.format(i)] = tranz['latitude'] - tranz['mecrh_lat_perc_{}'.format(i)]
    tranz['diff_lat_mecrh_long_perc_{}'.format(i)] = tranz['latitude'] - tranz['mecrh_long_perc_{}'.format(i)]
    #tranz['diff_lat_mecrh_lat_mean_perc_{}'.format(i)] = tranz['mecrh_lat_mean'] - tranz['mecrh_lat_perc_{}'.format(i)]
    #tranz['diff_lat_mecrh_long_mean_perc_{}'.format(i)] = tranz['mecrh_long_mean'] - tranz['mecrh_long_perc_{}'.format(i)]
    #tranz['diff_lat_mecrh_lat_med_perc_{}'.format(i)] = tranz['mecrh_lat_median'] - tranz['mecrh_lat_perc_{}'.format(i)]
    #tranz['diff_lat_mecrh_long_med_perc_{}'.format(i)] = tranz['mecrh_long_median'] - tranz['mecrh_long_perc_{}'.format(i)]
    #tranz['diff_lat_mecrh_lat_first_perc_{}'.format(i)] = tranz['mecrh_lat_first'] - tranz['mecrh_lat_perc_{}'.format(i)]
    #tranz['diff_lat_mecrh_long_first_perc_{}'.format(i)] = tranz['mecrh_long_first'] - tranz['mecrh_long_perc_{}'.format(i)]
    #tranz['diff_lat_mecrh_lat_last_perc_{}'.format(i)] = tranz['mecrh_lat_last'] - tranz['mecrh_lat_perc_{}'.format(i)]
    #tranz['diff_lat_mecrh_long_last_perc_{}'.format(i)] = tranz['mecrh_long_last'] - tranz['mecrh_long_perc_{}'.format(i)]
    

In [ ]:
for i in range(5,100,5):
    print(i)
    #tranz['diff_lat_mecrh_lat_mean_perc_{}'.format(i)] = tranz['mecrh_lat_mean'] - tranz['mecrh_lat_perc_{}'.format(i)]
    #tranz['diff_lat_mecrh_long_mean_perc_{}'.format(i)] = tranz['mecrh_long_mean'] - tranz['mecrh_long_perc_{}'.format(i)]
    #tranz['diff_lat_mecrh_lat_med_perc_{}'.format(i)] = tranz['mecrh_lat_median'] - tranz['mecrh_lat_perc_{}'.format(i)]
    #tranz['diff_lat_mecrh_long_med_perc_{}'.format(i)] = tranz['mecrh_long_median'] - tranz['mecrh_long_perc_{}'.format(i)]
    #tranz['diff_lat_mecrh_lat_first_perc_{}'.format(i)] = tranz['mecrh_lat_first'] - tranz['mecrh_lat_perc_{}'.format(i)]
    #tranz['diff_lat_mecrh_long_first_perc_{}'.format(i)] = tranz['mecrh_long_first'] - tranz['mecrh_long_perc_{}'.format(i)]
    #tranz['diff_lat_mecrh_lat_last_perc_{}'.format(i)] = tranz['mecrh_lat_last'] - tranz['mecrh_lat_perc_{}'.format(i)]
    #tranz['diff_lat_mecrh_long_last_perc_{}'.format(i)] = tranz['mecrh_long_last'] - tranz['mecrh_long_perc_{}'.format(i)]
    

In [ ]:
tranz['razmah_75_25_lat'] = tranz['mecrh_lat_perc_75'] - tranz['mecrh_lat_perc_25']
tranz['razmah_75_25_long'] = tranz['mecrh_long_perc_75'] - tranz['mecrh_long_perc_25']

In [ ]:
def divide(a, b):
    if b>0:
        return a/b
    else:
        return 8000#!!!!!!!!!!!!!!!!!!!!!!!!!!


In [ ]:
tranz['koef_oscil_lat'] = tranz.apply(lambda row: divide(row['max-min_lat'], row['mecrh_lat_mean']), axis=1)
tranz['koef_oscil_long'] = tranz.apply(lambda row: divide(row['max-min_long'], row['mecrh_long_mean']), axis=1)

In [ ]:
tranz['koef_oscil_lat_med'] = tranz.apply(lambda row: divide(row['max-min_lat'], row['mecrh_lat_median']), axis=1)
tranz['koef_oscil_long_med'] = tranz.apply(lambda row: divide(row['max-min_long'], row['mecrh_long_median']), axis=1)

In [ ]:
tranz['koef_var_lat'] = tranz.apply(lambda row: divide(row['mecrh_lat_std'], row['mecrh_lat_mean']), axis=1)
tranz['koef_var_long'] = tranz.apply(lambda row: divide(row['mecrh_long_std'], row['mecrh_long_mean']), axis=1)

In [ ]:
tranz['koef_var_lat_med'] = tranz.apply(lambda row: divide(row['mecrh_lat_std'], row['mecrh_lat_median']), axis=1)
tranz['koef_var_long_med'] = tranz.apply(lambda row: divide(row['mecrh_long_std'], row['mecrh_long_median']), axis=1)

In [ ]:
tranz['koef_quant_lat'] = tranz.apply(lambda row: divide(row['razmah_75_25_lat'], row['mecrh_lat_mean']), axis=1)
tranz['koef_quant_long'] = tranz.apply(lambda row: divide(row['razmah_75_25_lat'], row['mecrh_long_mean']), axis=1)

In [ ]:
tranz['koef_quant_lat_med'] = tranz.apply(lambda row: divide(row['razmah_75_25_lat'], row['mecrh_lat_median']), axis=1)
tranz['koef_quant_long_med'] = tranz.apply(lambda row: divide(row['razmah_75_25_lat'], row['mecrh_long_median']), axis=1)

In [ ]:
tranz.head()

In [ ]:
tranz.columns

In [ ]:
tranz['diff_lat_max'] = tranz['latitude'] - tranz['mecrh_lat_max']
tranz['diff_long_max'] = tranz['longitude'] - tranz['mecrh_long_max']
tranz['diff_lat_min'] = tranz['latitude'] - tranz['mecrh_lat_min']
tranz['diff_long_min'] = tranz['longitude'] - tranz['mecrh_long_min']
tranz['diff_lat_max+min'] = tranz['latitude'] - tranz['max+min_lat']
tranz['diff_long_max+min'] = tranz['longitude'] - tranz['max+min_long']
tranz['diff_lat_mecrh_lat_median'] = tranz['latitude'] - tranz['mecrh_lat_median']
tranz['diff_long_mecrh_long_median'] = tranz['longitude'] - tranz['mecrh_long_median']
tranz['diff_lat_mecrh_lat_first'] = tranz['latitude'] - tranz['mecrh_lat_first']
tranz['diff_long_mecrh_long_first'] = tranz['longitude'] - tranz['mecrh_long_first']
tranz['diff_lat_mecrh_lat_last'] = tranz['latitude'] - tranz['mecrh_lat_last']
tranz['diff_long_mecrh_long_last'] = tranz['longitude'] - tranz['mecrh_long_last']
tranz['diff_lat_mecrh_lat_mean'] = tranz['latitude'] - tranz['mecrh_lat_mean']
tranz['diff_long_mecrh_long_mean'] = tranz['longitude'] - tranz['mecrh_long_mean']
#tranz['diff_lat_'] = tranz['latitude'] - tranz['']
#tranz['diff_long_'] = tranz['longitude'] - tranz['']

tranz['mean-med_lat'] = (tranz['mecrh_lat_mean'] - tranz['mecrh_long_median'])
tranz['mean-med_long'] = tranz['mecrh_long_mean'] - tranz['mecrh_lat_median']
#tranz[''] = tranz[''] - tranz['']

In [38]:
def divide(a, b):
    if b>0:
        return a/b
    else:
        return 8000#!!!!!!!!!!!!!!!!!!!!!!!!!!
tranz['div_f_llllllllllllllllllllla'] = tranz.apply(lambda row: divide(row['mecrh_lat_first'], row['mecrh_lat_last']), axis=1)

In [2]:
tranz = pd.read_csv('data/tranz_prep.csv')
tranz.head()

,Test,latitude,longitude,merchant_id,x,y,cool,num,0,div,...,diff_lat_mecrh_lat_median,diff_long_mecrh_long_median,diff_lat_mecrh_lat_first,diff_long_mecrh_long_first,diff_lat_mecrh_lat_last,diff_long_mecrh_long_last,diff_lat_mecrh_lat_mean,diff_long_mecrh_long_mean,mean-med_lat,mean-med_long
0,0,55.74164,37.50263,267,55.726877,37.579645,0,13,2,6.5,...,0.024795,-0.077675,0.00000,0.00000,0.11028,0.03118,0.034329,-0.058564,18.127006,-18.155651
1,0,55.75035,37.62385,267,55.726877,37.579645,0,13,2,6.5,...,0.033505,0.043545,0.00871,0.12122,0.11899,0.15240,0.043039,0.062656,18.127006,-18.155651
2,0,55.72679,37.58025,267,55.726877,37.579645,1,13,2,6.5,...,0.009945,-0.000055,-0.01485,0.07762,0.09543,0.10880,0.019479,0.019056,18.127006,-18.155651
3,0,55.70701,37.58219,267,55.726877,37.579645,0,13,2,6.5,...,-0.009835,0.001885,-0.03463,0.07956,0.07565,0.11074,-0.000301,0.020996,18.127006,-18.155651
4,0,55.67474,37.55914,267,55.726877,37.579645,0,13,2,6.5,...,-0.042105,-0.021165,-0.06690,0.05651,0.04338,0.08769,-0.032571,-0.002054,18.127006,-18.155651


In [4]:
tranz.shape

TypeError: 'tuple' object is not callable

In [5]:
cor = np.array(tranz.corr())

In [13]:
np.savetxt('data/cor.txt',cor)

In [12]:
cor.savetxt()

AttributeError: 'numpy.ndarray' object has no attribute 'savetxt'

In [6]:
cor

array([[  1.00000000e+00,  -1.67853790e-02,   7.61505919e-03, ...,
         -3.60756949e-17,  -7.27467537e-03,   1.38136508e-02],
       [ -1.67853790e-02,   1.00000000e+00,  -2.92291776e-01, ...,
         -1.27843407e-01,   3.87637755e-01,  -4.23619007e-01],
       [  7.61505919e-03,  -2.92291776e-01,   1.00000000e+00, ...,
          6.68118861e-01,  -7.09006351e-01,   7.24272795e-01],
       ..., 
       [ -3.60756949e-17,  -1.27843407e-01,   6.68118861e-01, ...,
          1.00000000e+00,   3.07033029e-17,   2.20335021e-17],
       [ -7.27467537e-03,   3.87637755e-01,  -7.09006351e-01, ...,
          3.07033029e-17,   1.00000000e+00,  -9.71008433e-01],
       [  1.38136508e-02,  -4.23619007e-01,   7.24272795e-01, ...,
          2.20335021e-17,  -9.71008433e-01,   1.00000000e+00]])

In [9]:
len(cor)

227

In [ ]:
tranz.columns

In [3]:
tranz['dx'] = tranz['x'] - tranz['latitude']
tranz['dy'] = tranz['y'] - tranz['longitude']

In [ ]:
tranz['METALATsum']=tranz.groupby(['latitude'])['cool'].transform('sum')
tranz['METALATmean']=tranz.groupby(['latitude'])['cool'].transform('mean')
tranz['METAlongsum']=tranz.groupby(['longitude'])['cool'].transform('sum')
tranz['METAlongmean']=tranz.groupby(['longitude'])['cool'].transform('mean')

In [ ]:
tranz['countHourtr']=tranz.groupby(['Hourtr'])['Hourtr'].transform('count')
tranz['countminutetr']=tranz.groupby(['minutetr'])['minutetr'].transform('count')
tranz['countsecondtr']=tranz.groupby(['secondtr'])['secondtr'].transform('count')
tranz['countHourrec']=tranz.groupby(['Hourrec'])['Hourrec'].transform('count')
tranz['countminuterec']=tranz.groupby(['minuterec'])['minuterec'].transform('count')
tranz['countsecondrec']=tranz.groupby(['secondrec'])['secondrec'].transform('count')
tranz['countstr']=tranz.groupby(['str'])['str'].transform('count')
tranz['countsrec']=tranz.groupby(['srec'])['srec'].transform('count')
tranz['countsdiff']=tranz.groupby(['sdiff'])['sdiff'].transform('count')
tranz['countlat_diff_pos']=tranz.groupby(['latitude'])['dif_pos'].transform('sum')
tranz['countsum_sdfif']=abs(tranz.groupby(['latitude'])['sdiff'].transform('sum'))
tranz['countsum_abs_dif']=tranz.groupby(['latitude'])['sdiff'].transform('sum')
tranz['countlat_diff_posmean']=tranz.groupby(['latitude'])['dif_pos'].transform('mean')
tranz['countsum_sdfifmean']=abs(tranz.groupby(['latitude'])['sdiff'].transform('mean'))
tranz['countsum_abs_difmean']=tranz.groupby(['latitude'])['sdiff'].transform('mean')

In [3]:
def toch(x):
    return "%.0f" % x
tranz['lat'] = tranz['latitude'].apply(toch)
tranz['long'] = tranz['longitude'].apply(toch)
tranz['lat'] = tranz['lat'].apply(float)
tranz['long'] = tranz['long'].apply(float)

In [28]:
tranz = pd.get_dummies(tranz,columns = ['kmeans_70']).rename(columns=lambda x: str(x))

In [16]:
feat = ['latitude','longitude']

In [17]:
from sklearn.cluster import KMeans
for i in range(2,100):
    print(i)
    kmeans = KMeans(n_clusters=i,n_jobs = -1, random_state=i).fit(tranz[feat])
    tranz['kmeans_{}'.format(i)] = kmeans.labels_

2


/home/dz/.local/lib/python3.5/site-packages/sklearn/externals/joblib/hashing.py:197: DeprecationWarning: Changing the shape of non-C contiguous array by
descriptor assignment is deprecated. To maintain
the Fortran contiguity of a multidimensional Fortran
array, use 'a.T.view(...).T' instead
  obj_bytes_view = obj.view(self.np.uint8)
/home/dz/.local/lib/python3.5/site-packages/sklearn/externals/joblib/hashing.py:197: DeprecationWarning: Changing the shape of non-C contiguous array by
descriptor assignment is deprecated. To maintain
the Fortran contiguity of a multidimensional Fortran
array, use 'a.T.view(...).T' instead
  obj_bytes_view = obj.view(self.np.uint8)
/home/dz/.local/lib/python3.5/site-packages/sklearn/externals/joblib/hashing.py:197: DeprecationWarning: Changing the shape of non-C contiguous array by
descriptor assignment is deprecated. To maintain
the Fortran contiguity of a multidimensional Fortran
array, use 'a.T.view(...).T' instead
  obj_bytes_view = obj.view(self.np.ui

3


/home/dz/.local/lib/python3.5/site-packages/sklearn/externals/joblib/hashing.py:197: DeprecationWarning: Changing the shape of non-C contiguous array by
descriptor assignment is deprecated. To maintain
the Fortran contiguity of a multidimensional Fortran
array, use 'a.T.view(...).T' instead
  obj_bytes_view = obj.view(self.np.uint8)
/home/dz/.local/lib/python3.5/site-packages/sklearn/externals/joblib/hashing.py:197: DeprecationWarning: Changing the shape of non-C contiguous array by
descriptor assignment is deprecated. To maintain
the Fortran contiguity of a multidimensional Fortran
array, use 'a.T.view(...).T' instead
  obj_bytes_view = obj.view(self.np.uint8)
/home/dz/.local/lib/python3.5/site-packages/sklearn/externals/joblib/hashing.py:197: DeprecationWarning: Changing the shape of non-C contiguous array by
descriptor assignment is deprecated. To maintain
the Fortran contiguity of a multidimensional Fortran
array, use 'a.T.view(...).T' instead
  obj_bytes_view = obj.view(self.np.ui

4


...

/home/dz/.local/lib/python3.5/site-packages/sklearn/externals/joblib/hashing.py:197: DeprecationWarning: Changing the shape of non-C contiguous array by
descriptor assignment is deprecated. To maintain
the Fortran contiguity of a multidimensional Fortran
array, use 'a.T.view(...).T' instead
  obj_bytes_view = obj.view(self.np.uint8)
/home/dz/.local/lib/python3.5/site-packages/sklearn/externals/joblib/hashing.py:197: DeprecationWarning: Changing the shape of non-C contiguous array by
descriptor assignment is deprecated. To maintain
the Fortran contiguity of a multidimensional Fortran
array, use 'a.T.view(...).T' instead
  obj_bytes_view = obj.view(self.np.uint8)
/home/dz/.local/lib/python3.5/site-packages/sklearn/externals/joblib/hashing.py:197: DeprecationWarning: Changing the shape of non-C contiguous array by
descriptor assignment is deprecated. To maintain
the Fortran contiguity of a multidimensional Fortran
array, use 'a.T.view(...).T' instead
  obj_bytes_view = obj.view(self.np.ui

98


/home/dz/.local/lib/python3.5/site-packages/sklearn/externals/joblib/hashing.py:197: DeprecationWarning: Changing the shape of non-C contiguous array by
descriptor assignment is deprecated. To maintain
the Fortran contiguity of a multidimensional Fortran
array, use 'a.T.view(...).T' instead
  obj_bytes_view = obj.view(self.np.uint8)
/home/dz/.local/lib/python3.5/site-packages/sklearn/externals/joblib/hashing.py:197: DeprecationWarning: Changing the shape of non-C contiguous array by
descriptor assignment is deprecated. To maintain
the Fortran contiguity of a multidimensional Fortran
array, use 'a.T.view(...).T' instead
  obj_bytes_view = obj.view(self.np.uint8)
/home/dz/.local/lib/python3.5/site-packages/sklearn/externals/joblib/hashing.py:197: DeprecationWarning: Changing the shape of non-C contiguous array by
descriptor assignment is deprecated. To maintain
the Fortran contiguity of a multidimensional Fortran
array, use 'a.T.view(...).T' instead
  obj_bytes_view = obj.view(self.np.ui

99


/home/dz/.local/lib/python3.5/site-packages/sklearn/externals/joblib/hashing.py:197: DeprecationWarning: Changing the shape of non-C contiguous array by
descriptor assignment is deprecated. To maintain
the Fortran contiguity of a multidimensional Fortran
array, use 'a.T.view(...).T' instead
  obj_bytes_view = obj.view(self.np.uint8)
/home/dz/.local/lib/python3.5/site-packages/sklearn/externals/joblib/hashing.py:197: DeprecationWarning: Changing the shape of non-C contiguous array by
descriptor assignment is deprecated. To maintain
the Fortran contiguity of a multidimensional Fortran
array, use 'a.T.view(...).T' instead
  obj_bytes_view = obj.view(self.np.uint8)
/home/dz/.local/lib/python3.5/site-packages/sklearn/externals/joblib/hashing.py:197: DeprecationWarning: Changing the shape of non-C contiguous array by
descriptor assignment is deprecated. To maintain
the Fortran contiguity of a multidimensional Fortran
array, use 'a.T.view(...).T' instead
  obj_bytes_view = obj.view(self.np.ui

In [4]:
data_tr = tranz[tranz['Test']==0]
data_val = tranz[tranz['Test']==1]
tran = tranz[tranz['Test']!=2]
Xtest = tranz[tranz['Test']==2]
feat = list(set(data_tr.columns)-{'merchant_id','cool','x','y','dx','dy','meanlat','Test','METALATsum','METALATmean','METAlongsum','METAlongmean'})
print(data_tr.shape,data_val.shape,Xtest.shape)

(64663, 229) (15574, 229) (40967, 229)


In [18]:
tranz.head()

,Test,latitude,longitude,merchant_id,x,y,cool,num,0,div,...,kmeans_90,kmeans_91,kmeans_92,kmeans_93,kmeans_94,kmeans_95,kmeans_96,kmeans_97,kmeans_98,kmeans_99
0,0,55.74164,37.50263,267,55.726877,37.579645,0,13,2,6.5,...,2,32,52,39,13,10,1,0,0,2
1,0,55.75035,37.62385,267,55.726877,37.579645,0,13,2,6.5,...,2,32,52,39,13,10,1,0,0,2
2,0,55.72679,37.58025,267,55.726877,37.579645,1,13,2,6.5,...,2,32,52,39,13,10,1,0,0,2
3,0,55.70701,37.58219,267,55.726877,37.579645,0,13,2,6.5,...,2,32,52,39,13,10,1,0,0,2
4,0,55.67474,37.55914,267,55.726877,37.579645,0,13,2,6.5,...,2,32,52,39,13,10,87,0,64,84


In [ ]:
def transf_cat(train,test,feat_cat,feat_cont,fold):
    test['Proba'] = 0
    train['Proba'] = 0
    n_folds = fold
    kf = KFold(train.shape[0],n_folds)#,shuffle=True
    for train_index, val_index in kf:
        print("new_fold")
        training_data, validation_data = train.loc[train_index], train.loc[val_index]
        training_data['new_feat'] = training_data.groupby(feat_cat)[feat_cont].transform("mean")
        dat = training_data.groupby(feat_cat,as_index=False).mean()
        d={}
        for i in range(len(dat)):
            d[dat[feat_cat][i]] = dat['new_feat'][i]
        m=0#np.mean(training_data[feat_cont])
        def gr(x):
            try:
                return d[x]
            except:
                return m  
        train['Proba'].loc[val_index] = train[feat_cat].loc[val_index].apply(gr)
        test_new = test[feat_cat].apply(gr)
        test['Proba'] += np.array(test_new)
    return train['Proba'],test['Proba']*(1/n_folds)
from sklearn.cross_validation import KFold
time data_tr['latitude'+"n"] , data_val['latitude'+"n"] = transf_cat(data_tr,data_val,'latitude',"cool",20)

In [ ]:
time data_tr['latitude'+"n"] , data_val['latitude'+"n"] = transf_cat(data_tr,data_val,'latitude',"cool",20)

In [ ]:
#tranz.to_csv('data/tranz_prep.csv',index=False)

In [ ]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=2, random_state=0).fit(data_tr[feat])
kmeans.labels_
#kmeans.predict([[0, 0], [4, 4]])
#kmeans.cluster_centers_

In [40]:
Xtr, Xval, ytr, yval = data_tr.fillna(-999), data_val.fillna(-999), data_tr.cool, data_val.cool

In [ ]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=2, random_state=0).fit(Xtr[feat])
kmeans.labels_

In [ ]:
len(Xtr),len(kmeans.labels_)

In [ ]:
f = ['latitude','longitude']
f

In [ ]:
from sklearn.cluster import KMeans
for i in range(2,100):
    kmeans = KMeans(n_clusters=i,n_jobs = -1, random_state=i).fit(tranz[feat])
    tranz['kmeans_{}'.format(i)] = kmeans.labels_

In [5]:
Xtr, Xval, ytr, yval = data_tr.fillna(-999), data_val.fillna(-999), data_tr.dx, data_val.dx

# Train

# Model for x

In [6]:
dtr = xgb.DMatrix(Xtr[feat], label=ytr)
dval = xgb.DMatrix(Xval[feat], label=yval)
watchlist = [(dtr, 'train'), (dval, 'eval')]
history = dict()

In [8]:
params = {
    'max_depth': 7,
    'booster' :  'gbtree',
    'eta': 0.1,
    'nthread': 4,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'min_child_weight': 3,
    'seed':7
}
def xg_eval_popal(yhat, dtrain):
    y = dtrain.get_label()
    c = 500
    n = 2
    x = 500*np.abs(yhat-y)
    print('yhat',yhat)
    print('y',y)
    s = np.sum(1/(np.exp(-x**2)+1)-0.5)
    return 'errors',s

def fair_obj(preds, dtrain):
    n = 10
    с = 500
    labels = dtrain.get_label()
    x = с*np.abs(preds - labels)
    
    grad = (n*np.exp(-x**n)*(x**(n-1)))/(np.exp(-x**n)+1)**2
    hess = -(n*n*np.exp(-x**n)*(x**(2*n-2)))/(np.exp(-x**n)+1)**2+(2*n*n*np.exp(-2*x**n)*(x**(2*n-2)))/(np.exp(-x**n)+1)**3+((n-1)*n*np.exp(-x**n)*(x**(n-2)))/(np.exp(-x**n)+1)**2
    print('grad',grad)
    print('hess',hess)
    return grad, hess

model = xgb.train(params, dtr, num_boost_round=10, evals=watchlist,evals_result=history, verbose_eval=1,obj=fair_obj,feval=xg_eval_popal)

grad [ 0.  0.  0. ...,  0.  0.  0.]
hess [ nan  nan  nan ...,  nan  nan  nan]
yhat [ nan  nan  nan ...,  nan  nan  nan]
y [ -1.47630004e-02  -2.34730002e-02   8.70000003e-05 ...,   3.18149999e-02
   9.99350026e-02  -2.91249994e-02]
yhat [ nan  nan  nan ...,  nan  nan  nan]
y [-0.000477 -0.000447 -0.065115 ..., -0.002899 -0.249699 -0.157589]
[0]	train-errors:nan	eval-errors:nan
grad [ nan  nan  nan ...,  nan  nan  nan]
hess [ nan  nan  nan ...,  nan  nan  nan]
yhat [ nan  nan  nan ...,  nan  nan  nan]
y [ -1.47630004e-02  -2.34730002e-02   8.70000003e-05 ...,   3.18149999e-02
   9.99350026e-02  -2.91249994e-02]
yhat [ nan  nan  nan ...,  nan  nan  nan]
y [-0.000477 -0.000447 -0.065115 ..., -0.002899 -0.249699 -0.157589]
[1]	train-errors:nan	eval-errors:nan
grad [ nan  nan  nan ...,  nan  nan  nan]
hess [ nan  nan  nan ...,  nan  nan  nan]
yhat [ nan  nan  nan ...,  nan  nan  nan]
y [ -1.47630004e-02  -2.34730002e-02   8.70000003e-05 ...,   3.18149999e-02
   9.99350026e-02  -2.91249994e-

In [ ]:
params = {
    'max_depth': 7,
    'eta': 0.001,
    'nthread': 4,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'min_child_weight': 3,
    'seed':7
}
def xg_eval_popal(yhat, dtrain):
    y = dtrain.get_label()
    print('y',y)
    print('preds',yhat)
    x = 500*np.abs(y-yhat)
    s = np.sum(x*x)
    print('error',s)
    return 'error', s

def fair_obj(preds, dtrain):
    labels = dtrain.get_label()
    x = 500*np.abs(preds - labels)
    grad = 2*x
    print('grad',grad)
    hess = 2*np.ones(len(grad))
    print('hess',hess)
    return grad, hess
model = xgb.train(params, dtr, num_boost_round=10, evals=watchlist,evals_result=history, verbose_eval=1,obj=fair_obj,feval=xg_eval_popal)

In [ ]:
model = xgb.train(params, dtr, num_boost_round=300, evals=watchlist,evals_result=history, verbose_eval=50)

In [ ]:
pred = model.predict(dval)
sup = pd.DataFrame()
sup['merchant_id'] = Xval['merchant_id']
sup['latitude'] = Xval['latitude']
sup['x'] = Xval['x']
sup['pred'] = pred
sup.head()

In [ ]:
Xval.head()

In [ ]:
pred = model.predict(dval)
sup = pd.DataFrame()
sup['merchant_id'] = Xval['merchant_id']
sup['latitude'] = Xval['latitude']
sup['orig'] = sup['latitude']+Xval['x']
sup['x'] = Xval['x']
sup['pred'] = pred
sup['cool'] = Xval['cool']

In [ ]:
pred = model.predict(dval)
sup = pd.DataFrame()
sup['merchant_id'] = Xval['merchant_id']
sup['latitude'] = Xval['latitude']
sup['orig'] = sup['latitude']+Xval['x']
sup['x'] = Xval['x']
sup['pred'] = pred
sup['cool'] = Xval['cool']
sup = sup.groupby('merchant_id',as_index=False).mean()
sup['x_p'] = abs(sup['x']-sup['pred'])
def bol(x):
    if x<0.002:
        return 1
    else:
        return 0
sup['x_p'] = sup['x_p'].apply(bol)
np.mean(sup['x_p'])

In [ ]:
np.mean(sup['x_p'])

In [ ]:
sup.head()

In [ ]:
sup['latitude'] = sup['latitude'] + sup['pred']
sup['x_p'] = abs(sup['orig']-sup['latitude'])
def bol(x):
    if x<0.002:
        return 1
    else:
        return 0
sup['x_p'] = sup['x_p'].apply(bol)

In [13]:
Xtr[feat].shape

(64663, 367)

# model for 0,1( bad/good )

In [41]:
dtr = xgb.DMatrix(Xtr[feat], label=ytr)
dval = xgb.DMatrix(Xval[feat], label=yval)
dtest = xgb.DMatrix(Xtest[feat])
watchlist = [(dtr, 'train'), (dval, 'eval')]
history = dict()

In [ ]:
binary:logitraw
    'binary:logistic',

In [42]:
params = {
    'max_depth': 5,
    'eta': 0.025,
    'nthread': 4,
    'subsample': 0.8,
    'eval_metric': ['auc'],
    'objective': 'binary:logistic',
    'colsample_bytree': 0.8,
    'min_child_weight': 100,
    #'scale_pos_weight':(1-tran.cool.mean())/tran.cool.mean(),
    'seed':7
}

In [ ]:
data_tr['cool'].mean()

In [ ]:
xgb.cv(params, dtr, 1000, nfold=5,
       metrics={'auc'}, seed = 7,verbose_eval=50,
       callbacks=[xgb.callback.print_evaluation(show_stdv=True)])

In [43]:
model = xgb.train(params, dtr, num_boost_round=1000, evals=watchlist,evals_result=history, verbose_eval=50)

[0]	train-auc:0.906544	eval-auc:0.899022
[50]	train-auc:0.936419	eval-auc:0.929387
[100]	train-auc:0.941324	eval-auc:0.934667
[150]	train-auc:0.947033	eval-auc:0.940946
[200]	train-auc:0.951912	eval-auc:0.944665
[250]	train-auc:0.956005	eval-auc:0.947379
[300]	train-auc:0.958877	eval-auc:0.948954
[350]	train-auc:0.960917	eval-auc:0.950065
[400]	train-auc:0.962834	eval-auc:0.950837
[450]	train-auc:0.964553	eval-auc:0.95147
[500]	train-auc:0.965948	eval-auc:0.951976
[550]	train-auc:0.967029	eval-auc:0.952082
[600]	train-auc:0.967947	eval-auc:0.952256
[650]	train-auc:0.968817	eval-auc:0.95259
[700]	train-auc:0.969621	eval-auc:0.952887
[750]	train-auc:0.970486	eval-auc:0.953021
[800]	train-auc:0.971203	eval-auc:0.953174
[850]	train-auc:0.971861	eval-auc:0.953336
[900]	train-auc:0.972589	eval-auc:0.953394
[950]	train-auc:0.973133	eval-auc:0.95353


In [ ]:
from sklearn.metrics import recall_score
def xgb_eval(yhat, dtrain):
    y = dtrain.get_label()
    for i 
    return 'recall', recall_score(y, yhat, average='macro')
model = xgb.train(params, dtr, num_boost_round=1000, evals=watchlist,evals_result=history, verbose_eval=50,feval=xgb_eval)

In [ ]:
700 - 0945
0956
0957
0959 mean-proc,first-proc///

In [44]:
xgb.plot_importance(model,title='Feature importance',  height=0.1,  xlabel='F score', ylabel='Features',  importance_type='gain',grid=False, )
plt.show()

In [ ]:
imps = pd.Series(model.get_fscore()).sort_values()
imps.plot(kind='bar', figsize=(15,6),logy=False)
plt.show()

In [45]:
pred = model.predict(dval)
#pred = lr.predict_proba(Xval[feat])[0:len(Xval),1]
Xv = Xval.copy()
Xv['p'] = pred
Xv['pro']=Xv.groupby(['merchant_id'])['p'].transform('max')
Xv.head()

,Test,latitude,longitude,merchant_id,x,y,cool,num,0,div,...,diff_long_mecrh_long_first,diff_lat_mecrh_lat_last,diff_long_mecrh_long_last,diff_lat_mecrh_lat_mean,diff_long_mecrh_long_mean,mean-med_lat,mean-med_long,div_f_llllllllllllllllllllla,p,pro
64663,1,55.05603,82.91273,178,55.055553,82.911833,1,8,4,2,...,0.00000,0.00003,-0.00026,0.000015,-0.000130,-27.856845,27.856845,1.000001,0.374649,0.492466
64664,1,55.05600,82.91299,178,55.055553,82.911833,1,8,4,2,...,0.00026,0.00000,0.00000,-0.000015,0.000130,-27.856845,27.856845,1.000001,0.492466,0.492466
64665,1,55.81760,37.71386,1509,55.752485,37.623196,0,5,1,5,...,0.00000,0.06478,0.09474,0.070437,0.096667,18.128043,-18.135627,1.001162,0.094817,0.094817
64666,1,55.67107,37.51860,1509,55.752485,37.623196,0,5,1,5,...,-0.19526,-0.08175,-0.10052,-0.076093,-0.098593,18.128043,-18.135627,1.001162,0.006571,0.094817
64667,1,55.75282,37.61912,1509,55.752485,37.623196,0,5,1,5,...,-0.09474,0.00000,0.00000,0.005657,0.001927,18.128043,-18.135627,1.001162,0.015182,0.094817


In [46]:
Xv['maxim_prob'] = Xv['pro'] - Xv['p']
print(np.mean(Xv[Xv['maxim_prob']==0]['cool']))
Xv[Xv['maxim_prob']==0].head()

0.3452566096423017


,Test,latitude,longitude,merchant_id,x,y,cool,num,0,div,...,diff_lat_mecrh_lat_last,diff_long_mecrh_long_last,diff_lat_mecrh_lat_mean,diff_long_mecrh_long_mean,mean-med_lat,mean-med_long,div_f_llllllllllllllllllllla,p,pro,maxim_prob
64664,1,55.05600,82.91299,178,55.055553,82.911833,1,8,4,2,...,0.00000,0.00000,-0.000015,0.000130,-27.856845,27.856845,1.000001,0.492466,0.492466,0
64665,1,55.81760,37.71386,1509,55.752485,37.623196,0,5,1,5,...,0.06478,0.09474,0.070437,0.096667,18.128043,-18.135627,1.001162,0.094817,0.094817,0
64673,1,59.85780,30.38800,2049,59.857468,30.388245,1,21,1,21,...,0.00007,-0.00277,0.612839,-1.115905,28.856761,-28.355935,1.000195,0.634429,0.634429,0
64683,1,55.80821,37.51331,4262,55.808543,37.512389,1,24,1,24,...,0.03598,-0.07933,0.014156,-0.031625,18.278109,-18.261150,0.998898,0.787597,0.787597,0
64702,1,55.77462,37.65085,6422,55.776062,37.649795,1,10,0,8000,...,-0.00022,0.00107,0.033812,0.026438,18.090108,-18.150208,0.996962,0.765152,0.765152,0


In [25]:
Xv['prod'] = Xv['latitude']*Xv['p']*Xv['p']*Xv['p']
Xv['p1'] = Xv['p']*Xv['p']*Xv['p']
Xv['prob_sum'] = Xv.groupby(['merchant_id'])['p1'].transform('sum')
Xv['prod_sum'] = Xv.groupby(['merchant_id'])['prod'].transform('sum')
def divide(a, b):
    if b>0:
        return a/b
    else:
        return 8000#!!!!!!!!!!!!!!!!!!!!!!!!!!
Xv['preds_x'] = Xv.apply(lambda row: divide(row['prod_sum'], row['prob_sum']), axis=1)

sup = Xv.groupby('merchant_id',as_index=False).mean()

In [ ]:
sup.head()

In [ ]:
Xv['prod'] = np.log(Xv['latitude']*Xv['p'])
Xv['p1'] = Xv['p']

In [ ]:
Xv['prob_sum'] = Xv.groupby(['merchant_id'])['p1'].transform('sum')

In [ ]:
Xv['prod_sum'] = Xv.groupby(['merchant_id'])['prod'].transform('sum')

In [ ]:
len(Xv['prob_sum']),len(Xv['prod_sum'])

In [ ]:
def divide(a, b):
    if b>0:
        return a/b
    else:
        return 8000#!!!!!!!!!!!!!!!!!!!!!!!!!!
Xv['preds'] = Xv.apply(lambda row: divide(row['prod_sum'], row['prob_sum']), axis=1)
#Xv['preds'] = np.exp(Xv['preds'])

In [ ]:
sup = Xv.groupby('merchant_id',as_index=False).mean()
sup['x_p'] = abs(sup['x']-sup['preds'])
def bol(x):
    if x<0.002:
        return 1
    else:
        return 0
sup['x_p'] = sup['x_p'].apply(bol)
np.mean(sup['x_p'])

In [ ]:
Xv['maxim_prob'] = Xv['pro'] - Xv['p']
print(np.mean(Xv[Xv['maxim_prob']==0]['cool']))
Xv[Xv['maxim_prob']==0].head()

In [ ]:
import scipy.stats as stats
stats.pearsonr(pred,pred1),stats.spearmanr(pred,pred1),stats.kendalltau(pred,pred1)

In [ ]:
import scipy.stats as stats
stats.pearsonr(pred,pred1),stats.spearmanr(pred,pred1),stats.kendalltau(pred,pred1)

In [ ]:
0.342
0359
0363 oscil
035

In [ ]:
dtrain = xgb.DMatrix(tran[feat], label=tran.cool)
dtest = xgb.DMatrix(Xtest[feat])

In [ ]:
model = xgb.train(params, dtrain, num_boost_round=900, evals=watchlist,evals_result=history, verbose_eval=50)


In [ ]:
pred_test = model.predict(dtest)

In [ ]:
Xtest['prod'] = Xtest['latitude']*pred_test*pred_test*pred_test
Xtest['p1'] = pred_test*pred_test*pred_test
Xtest['prob_sum'] = Xtest.groupby(['merchant_id'])['p1'].transform('sum')
Xtest['prod_sum'] = Xtest.groupby(['merchant_id'])['prod'].transform('sum')
def divide(a, b):
    if b>0:
        return a/b
    else:
        return 8000#!!!!!!!!!!!!!!!!!!!!!!!!!!
Xtest['preds_x'] = Xtest.apply(lambda row: divide(row['prod_sum'], row['prob_sum']), axis=1)
Xtest['prod'] = Xtest['longitude']*pred_test*pred_test*pred_test
Xtest['p1'] = pred_test*pred_test*pred_test
Xtest['prob_sum'] = Xtest.groupby(['merchant_id'])['p1'].transform('sum')
Xtest['prod_sum'] = Xtest.groupby(['merchant_id'])['prod'].transform('sum')
Xtest['preds_y'] = Xtest.apply(lambda row: divide(row['prod_sum'], row['prob_sum']), axis=1)
Xt = Xtest.groupby('merchant_id',as_index=False).mean()
Xt = Xt.drop_duplicates(cols=['merchant_id'], take_last=True)
s = pd.DataFrame()
s['_ID_'] = Xt['merchant_id']
s['_LAT_'] = Xt['preds_x']
s['_LON_'] = Xt['preds_y']

In [ ]:
Xtest['p'] = pred_test
Xtest['pro'] = Xtest.groupby(['merchant_id'])['p'].transform('max')
Xtest['maxim_prob'] = Xtest['pro'] - Xtest['p']
Xt = Xtest[Xtest['maxim_prob']==0]
Xt = Xt.drop_duplicates(cols=['merchant_id'], take_last=True)
s = pd.DataFrame()
s['_ID_'] = Xt['merchant_id']
s['_LAT_'] = Xt['latitude']
s['_LON_'] = Xt['longitude']
s.head()

In [ ]:
len(s)

In [ ]:
dif = set(test.merchant_id) - set(s._ID_)
list(dif)

In [ ]:
dif = set(s._ID_) - set(test.merchant_id) 
dif

In [ ]:
add = pd.DataFrame()
add['_ID_'] = list(dif)
add['_LAT_'] = 0
add['_LON_'] = 0

In [ ]:
len(test.merchant_id),len(s._ID_)+len(add._ID_)

In [ ]:
sub = pd.concat([s,add],axis = 0)
sub.tail(20)

In [ ]:
train.isnull().sum()

In [ ]:
len(sub)

In [ ]:
min(tranz.num)

In [ ]:
len(Xt),len(test),len(s)

In [ ]:
Xv = Xval.copy()
Xv['p'] = pred
Xv['pro']=Xv.groupby(['merchant_id'])['p'].transform('max')
Xv.head()

In [ ]:
Xv['pro']=Xv.groupby(['merchant_id'])['p'].transform('max')
Xv.head()

In [ ]:
x1 = 1.003
x2 = 1
p1 = 0.1
p2 = 0.2
(x1*p1+x2*p2)/(p1+p2)

In [ ]:
Xv['maxim_prob'] = Xv['pro'] - Xv['p']
X = Xv[Xv['maxim_prob']==0]
print(np.mean(X['cool']))
X.head()

In [ ]:
X = Xv[Xv['maxim_prob']==0]
X.head()

In [ ]:
X1 = Xv[Xv['maxim_prob']!=0]
X1['pro']=X1.groupby(['merchant_id'])['p'].transform('max')
X1['maxim_prob'] = X1['pro'] - X1['p']
X1 = X1[X1['maxim_prob']==0]
np.mean(X1['cool'])

In [ ]:
np.mean(X['cool'])

In [ ]:
np.mean(Xv['pr'])

In [ ]:
len(pred),len(yval)

In [ ]:
sub = pd.merge(test,mlatitude,on='merchant_id')

In [ ]:
len(sub),len(test)

In [ ]:
sub.head()

In [ ]:
s = pd.DataFrame()
s['_ID_'] = test['merchant_id']
s['_LAT_'] = sub['latitude_y']
s['_LON_'] = sub['longitude_y']
s.head()

In [ ]:
test.head()

In [ ]:
sub.to_csv('pred2/xgb900_inrussia0953(39)otbormean.csv',index=False)